In [4]:
from google.colab import files
uploaded = files.upload()


Saving medical_data.csv to medical_data (1).csv
Saving Train_data.csv to Train_data (1).csv


In [5]:
df1 = pd.read_csv("Train_data (1).csv")[['text', 'label']]


In [6]:
df2 = pd.read_csv("medical_data.csv").rename(columns={"Patient_Problem": "text", "Disease": "label"})[['text', 'label']]


In [8]:
# 📦 1. Installieren von notwendigen Paketen
!pip install transformers scikit-learn pandas torch -q

# 📂 2. Datensätze laden und zusammenführen
import pandas as pd

# Lade deine CSV-Dateien (nachdem du sie hochgeladen hast!)
df1 = pd.read_csv("Train_data.csv")[['text', 'label']]
df2 = pd.read_csv("medical_data.csv").rename(columns={"Patient_Problem": "text", "Disease": "label"})[['text', 'label']]
df_all = pd.concat([df1, df2]).reset_index(drop=True)

# 🔠 3. Labels encodieren
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_all['label'] = le.fit_transform(df_all['label'])
num_labels = len(le.classes_)

# 🔀 4. Daten aufteilen
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_all['text'], df_all['label'], test_size=0.2, random_state=42)

# 🧾 5. Tokenizer und Dataset-Klasse
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader

model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

class MedicalDataset(Dataset):
    def __init__(self, texts, labels):
        encodings = tokenizer(texts.tolist(), truncation=True, padding=True, return_tensors="pt")
        self.encodings = encodings
        self.labels = torch.tensor(labels.tolist())

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# 📥 6. Test-Set vorbereiten
test_dataset = MedicalDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=16)

# 🤖 7. Modell laden (ohne Fine-Tuning)
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 🧪 8. Evaluation ohne Fine-Tuning
from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np

model.eval()
preds, labels = [], []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = model(**batch).logits
        preds.extend(torch.argmax(logits, axis=1).cpu().numpy())
        labels.extend(batch["labels"].cpu().numpy())

# ✅ 9. Metriken nur für vorkommende Labels
used_labels = np.unique(labels)
used_names = le.inverse_transform(used_labels)

print("\n📊 PubMedBERT WITHOUT Fine-Tuning:")
print(classification_report(labels, preds, labels=used_labels, target_names=used_names))
print("Accuracy:", accuracy_score(labels, preds))
print("F1 Score:", f1_score(labels, preds, average="weighted"))



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



📊 PubMedBERT WITHOUT Fine-Tuning:
                                          precision    recall  f1-score   support

                     Achilles Tendinitis       0.00      0.00      0.00         1
                                    Acne       0.00      0.00      0.00        14
        Age-related Macular Degeneration       0.00      0.00      0.00         1
                             Anaphylaxis       0.00      0.00      0.00         1
                                Anorexia       0.00      0.00      0.00         1
                               Arthritis       0.00      0.00      0.00        14
                                  Asthma       0.00      0.00      0.00         1
Attention Deficit Hyperactivity Disorder       0.00      0.00      0.00         1
                        Bronchial Asthma       0.00      0.00      0.00         7
                                COVID-19       0.00      0.00      0.00         1
                    Cervical spondylosis       0.00      0.00 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
import numpy as np

# Modell ist bereits eval (nicht trainiert)
model.eval()
preds, labels = [], []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = model(**batch).logits
        preds.extend(torch.argmax(logits, axis=1).cpu().numpy())
        labels.extend(batch["labels"].cpu().numpy())

# 🔍 Nur vorkommende Klassen im Testset
used_labels = np.unique(labels)
used_names = le.inverse_transform(used_labels)

# 📊 Klassifikationsreport + Einzelmetriken
print("\n📊 Evaluation WITHOUT Fine-Tuning:")
print(classification_report(labels, preds, labels=used_labels, target_names=used_names))

# 📌 Einzelmetriken zusätzlich separat
print("🔢 Accuracy: ", round(accuracy_score(labels, preds), 4))
print("🎯 Precision (weighted): ", round(precision_score(labels, preds, average="weighted"), 4))
print("📈 Recall (weighted): ", round(recall_score(labels, preds, average="weighted"), 4))
print("📊 F1 Score (weighted): ", round(f1_score(labels, preds, average="weighted"), 4))



📊 Evaluation WITHOUT Fine-Tuning:
                                          precision    recall  f1-score   support

                     Achilles Tendinitis       0.00      0.00      0.00         1
                                    Acne       0.00      0.00      0.00        14
        Age-related Macular Degeneration       0.00      0.00      0.00         1
                             Anaphylaxis       0.00      0.00      0.00         1
                                Anorexia       0.00      0.00      0.00         1
                               Arthritis       0.00      0.00      0.00        14
                                  Asthma       0.00      0.00      0.00         1
Attention Deficit Hyperactivity Disorder       0.00      0.00      0.00         1
                        Bronchial Asthma       0.00      0.00      0.00         7
                                COVID-19       0.00      0.00      0.00         1
                    Cervical spondylosis       0.00      0.00 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m